<a href="https://colab.research.google.com/github/amitav710/FlickIt-Assgn/blob/main/FlickitAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Download YOLOv3 weights
!wget https://pjreddie.com/media/files/yolov3.weights

# Download YOLOv3 configuration file
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg

--2024-05-26 16:46:34--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  47.3MB/s    in 4.9s    

2024-05-26 16:46:39 (48.1 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2024-05-26 16:46:40--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2

In [4]:
# Install OpenCV, NumPy, and PyTube
!pip install opencv-python-headless numpy pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00


In [5]:
# Install MediaPipe
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [ ]:
import cv2
import numpy as np
from pytube import YouTube
import os
from google.colab.patches import cv2_imshow
import mediapipe as mp

# Function to download video from YouTube
def download_youtube_video(url, output_path='video.mp4'):
    yt = YouTube(url)
    stream = yt.streams.filter(file_extension='mp4').first()
    stream.download(filename=output_path)
    return output_path

# Function to check if the estimated toe position is above the y-coordinate of the ball's bounding box
# and within the x-range of the ball's bounding box
def is_valid_touch(ball_box, toe_pos):
    x, y, w, h = ball_box
    if toe_pos[1] < y and x <= toe_pos[0] <= x + w:
        return True
    return False

# Function to calculate velocity
def calculate_velocity(prev_pos, current_pos, factor=1):
    return np.linalg.norm(np.array(current_pos) - np.array(prev_pos)) / factor

# Function to determine ball spin direction using optical flow
def calculate_spin_direction(prev_frame, current_frame, ball_bbox):
    x, y, w, h = ball_bbox
    prev_ball = prev_frame[y:y+h, x:x+w]
    curr_ball = current_frame[y:y+h, x:x+w]
    prev_gray = cv2.cvtColor(prev_ball, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr_ball, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    avg_flow_y = np.mean(flow[..., 1])
    if avg_flow_y > 0:
        return 'Forward'
    else:
        return 'Backward'

# Download the video from YouTube
video_url = 'https://youtu.be/k9gRgg_tW24'  # Replace with your YouTube URL
video_path = download_youtube_video(video_url)

# Initialize variables for touch count, player velocity, and ball movement direction
right_leg_touches = 0
left_leg_touches = 0
player_velocity = []
ball_movement_direction = "Stationary"
prev_ball_pos = None
prev_frame = None
last_right_touch_frame = -10
last_left_touch_frame = -10
prev_right_ankle_pos = None
prev_left_ankle_pos = None
velocity_threshold = 2.0  # Threshold to avoid false touches

# Paths to YOLO configuration and weights files
yolo_cfg_path = 'yolov3.cfg'
yolo_weights_path = 'yolov3.weights'

# Pre-trained model for object detection
model = cv2.dnn.readNetFromDarknet(yolo_cfg_path, yolo_weights_path)
layer_names = model.getLayerNames()
output_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Process from 40 seconds to 70 seconds and every 3rd frame
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
start_frame = int(40 * fps)
end_frame = int(70 * fps)
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
frame_count = start_frame
prev_pos = None
velocity_smooth = 0

while cap.isOpened() and frame_count < end_frame:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 3 != 0:
        continue

    height, width, channels = frame.shape

    # Object detection
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    model.setInput(blob)
    outs = model.forward(output_layers)

    ball_box = None
    current_ball_pos = None
    current_pos = None

    # Analyze detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = center_x - w // 2
                y = center_y - h // 2

                if class_id == 32:  # Sports Ball
                    ball_box = (x, y, w, h)
                    current_ball_pos = (center_x, center_y)

                if class_id == 0:  # Person
                    current_pos = (center_x, center_y)

    # Determine ball movement direction using optical flow
    if prev_frame is not None and ball_box is not None:
        ball_movement_direction = calculate_spin_direction(prev_frame, frame, ball_box)

    prev_ball_pos = current_ball_pos
    prev_frame = frame.copy()

    # Pose estimation for legs detection
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        right_ankle_pos = (int(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].x * width),
                         int(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y * height))
        left_ankle_pos = (int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x * width),
                        int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y * height))

        if ball_box and current_ball_pos:
            right_toe_pos = ((right_ankle_pos[0] + current_ball_pos[0]) // 2,
                             (right_ankle_pos[1] + current_ball_pos[1]) // 2)
            left_toe_pos = ((left_ankle_pos[0] + current_ball_pos[0]) // 2,
                            (left_ankle_pos[1] + current_ball_pos[1]) // 2)

            # Calculate velocities for the ankles
            right_ankle_velocity = calculate_velocity(prev_right_ankle_pos, right_ankle_pos, factor=3) if prev_right_ankle_pos else 0
            left_ankle_velocity = calculate_velocity(prev_left_ankle_pos, left_ankle_pos, factor=3) if prev_left_ankle_pos else 0

            if is_valid_touch(ball_box, right_toe_pos) and frame_count - last_right_touch_frame > 3 and right_ankle_velocity > velocity_threshold:
                right_leg_touches += 1
                last_right_touch_frame = frame_count
            if is_valid_touch(ball_box, left_toe_pos) and frame_count - last_left_touch_frame > 3 and left_ankle_velocity > velocity_threshold:
                left_leg_touches += 1
                last_left_touch_frame = frame_count

            prev_right_ankle_pos = right_ankle_pos
            prev_left_ankle_pos = left_ankle_pos

    # Calculate player's velocity (smoothed)
    if prev_pos and current_pos:
        velocity = calculate_velocity(prev_pos, current_pos, factor=3)
        velocity_smooth = 0.9 * velocity_smooth + 0.1 * velocity  # Exponential moving average
        player_velocity.append(velocity_smooth)
        prev_pos = current_pos
    else:
        prev_pos = current_pos
        player_velocity.append(0)

    # Overlay the results on the frame
    cv2.putText(frame, f"Right Leg Touches: {right_leg_touches}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f"Left Leg Touches: {left_leg_touches}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Player Velocity: {player_velocity[-1]:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"Ball Movement: {ball_movement_direction}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # Display the frame
    cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()